# TEXT TOKENIZATION

# 1 -From text to tokens

Language models like DistilBERT cannot receive raw strings as input; instead, they assume the text has been *tokenized* and *encoded* as numerical vectors. Tokenization is the step of breaking down a string into the atomic units used in the model. There are several tokenization strategies one can adopt, and the optimal splitting of words into subunits is usually learned from the corpus. Before looking at the tokenizer used for DistilBERT, let's consider two extreme cases: *character* and *word* tokenization.

## 1.1 - Character tokenization


The simplest tokenization scheme is to feed each character individually to the model. In Python, `str` objects are really arrays under the hood, which allows us to quickly implement character-level tokenization with just one line of code:

In [48]:
text = "Tokenizing text is a core task of NLP !"
tokenized_text = list(text)
print(tokenized_text)

['T', 'o', 'k', 'e', 'n', 'i', 'z', 'i', 'n', 'g', ' ', 't', 'e', 'x', 't', ' ', 'i', 's', ' ', 'a', ' ', 'c', 'o', 'r', 'e', ' ', 't', 'a', 's', 'k', ' ', 'o', 'f', ' ', 'N', 'L', 'P', ' ', '!']


This is a good start, but we are not done yet. Our model expects each character to be converted to an integer, a process sometimes called *numericalization*. One simple way to do this is by encoding each unique token (which are characters in this case) with a unique integer:

In [49]:
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)

{' ': 0, '!': 1, 'L': 2, 'N': 3, 'P': 4, 'T': 5, 'a': 6, 'c': 7, 'e': 8, 'f': 9, 'g': 10, 'i': 11, 'k': 12, 'n': 13, 'o': 14, 'r': 15, 's': 16, 't': 17, 'x': 18, 'z': 19}


This gives us a mapping from each character in our vocabulary to a unique integer. We can now use `token2idx` to transform the tokenized text to a list of integers:

In [50]:
input_ids = [token2idx[token] for token in tokenized_text]
print(input_ids)

[5, 14, 12, 8, 13, 11, 19, 11, 13, 10, 0, 17, 8, 18, 17, 0, 11, 16, 0, 6, 0, 7, 14, 15, 8, 0, 17, 6, 16, 12, 0, 14, 9, 0, 3, 2, 4, 0, 1]


Each token has now been mapped to a unique numerical identifier (hence the name `input_ids`). The last step is to convert `input_ids` to a 2D tensor of one-hot vectors. One-hot vectors are frequently used in machine learning to encode categorical data, which can be either ordinal or nominal. 

The problem with character-level tokenization ignores any structure in the text and treats the whole string as a stream of characters. Although this helps deal with misspellings and rare words, **the main drawback is that linguistic structures such as words need to be learned from the data**. This requires significant
compute, memory, and data. For this reason, character tokenization is rarely used in practice.

## 1.2 - Word tokenization

Instead of splitting the text into characters, we can split it into words and map each word to an integer. Using words from the outset enables the model to skip the step of learning words from characters, and thereby reduces the complexity of the training process.

One simple class of word tokenizers uses whitespace to tokenize the text. We can do this by applying Python's `split()` function directly:

In [51]:
tokenized_text = text.split()
print(tokenized_text)

['Tokenizing', 'text', 'is', 'a', 'core', 'task', 'of', 'NLP', '!']


From here we can take the same steps we took for the character tokenizer to map each word to an ID. However, we can already see one potential problem with this tokenization scheme: punctuation is not accounted for, so `NLP.` is treated as a single token. Given that words can include declinations, conjugations, or misspellings, the size of the vocabulary can easily grow into the millions!

----

<mark><b>Note:</b></mark> Some word tokenizers have extra rules for punctuation. One can also apply stemming or lemmatization, which normalizes words to their stem (e.g., "great", "greater", and "greatest" all become "great"), at the expense of losing some information in the text.
    
----
    
<span style="color:blue"><b>Having a large vocabulary is a problem because it requires neural networks to have an enormous number of parameters</b></span>. To illustrate this, suppose we have 1 million unique words and want to compress the 1-million-dimensional input vectors to 1-thousanddimensional vectors in the first layer of our neural network. This is a standard step in most NLP architectures, and **the resulting weight matrix of this first layer would contain 1 million × 1 thousand = 1 billion weights**. This is already comparable to the largest GPT-2 model, which has around 1.5 billion parameters in total!
    
Naturally, we want to avoid being so wasteful with our model parameters since models are expensive to train, and larger models are more difficult to maintain. <span style="color:blue"><b>A common approach is to limit the vocabulary and discard rare words</b></span> by considering, say, the 100,000 most common words in the corpus. Words that are not part of the vocabulary are classified as "unknown" and mapped to a shared `UNK` token. This means that we lose some potentially important information in the process of word tokenization, since the model has no information about words associated with `UNK`.

## 1.3 - Subword tokenization

The basic idea behind subword tokenization is to combine the best aspects of character and word tokenization. On the one hand, we want to split rare words into smaller units to allow the model to deal with complex words and misspellings. On the other hand, we want to keep frequent words as unique entities so that we can keep the length of our inputs to a manageable size. The main distinguishing feature of subword tokenization (as well as word tokenization) is that it is learned from the pretraining corpus using a mix of statistical rules and algorithms.

[There are several subword tokenization algorithms that are commonly used in NLP:](https://huggingface.co/docs/transformers/tokenizer_summary)
* Byte-Pair Encoding tokenizer.
* WordPiece tokenizer.
* Unigram tokenizer.
* SentencePiece tokenizer.

### 1.3.1 - Byte-Pair Encoding tokenizer

Byte-Pair Encoding (BPE) was initially developed as an algorithm to compress texts, and then used by OpenAI for tokenization when pretraining the GPT model. It’s used by a lot of Transformer models, including GPT, GPT-2, RoBERTa, BART, and DeBERTa. [🤗 NLP course provides a great article explaining it](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt).

Main aspects of BPE:

* **Vocabulary building process**. It starts with individual characters (i.e.,  all the ASCII characters and probably some Unicode characters as well.) as the initial vocabulary. After getting this base vocabulary, the algorithm iteratively merges the most frequent character pairs, gradually building a vocabulary of subword units. This merging continues (with increasing subword size) until the desired vocabulary size is reached, resulting in a comprehensive set of subword units that represent frequent and rare combinations of characters.

* **Tokenization algorithm**. Once the BPE vocabulary is created, the tokenization process involves splitting words into subword units based on the learned vocabulary. Words are scanned from left to right, and the longest matching subword unit is identified at each step. The identified subword unit is replaced with a special token, preserving the word structure. This tokenization captures the internal subword structure of words, allowing effective handling of out-of-vocabulary terms and improving the model's ability to represent complex words.

To better understand WordPiece, we are going to see it in action. Thankfully, Transformers provides a convenient AutoTokenizer class that allows you to quickly load the tokenizer associated with a pretrained model by providing the ID of the model in the Hub or a local file path:

In [52]:
from transformers import AutoTokenizer

gpt2_model_ckpt = "gpt2"
bpe_tokenizer = AutoTokenizer.from_pretrained(gpt2_model_ckpt)

The `AutoTokenizer` class belongs to a larger set of “auto” classes whose job is to automatically retrieve the model’s configuration, pretrained weights, or vocabulary from the name of the checkpoint. This allows you to quickly switch between models, but if you wish to load the specific class manually you can do so as well. For example, we could have loaded the **GPT2** tokenizer as follows:

In [53]:
from transformers import GPT2Tokenizer

gpt2_tokenizer = GPT2Tokenizer.from_pretrained(gpt2_model_ckpt)

---

<mark><b>Note:</mark> When you run the `AutoTokenizer.from_pretrained()` method for the first time you will see a progress bar that shows which parameters of the pretrained tokenizer are loaded from the Hugging
Face Hub. When you run the code a second time, it will load the tokenizer from the cache, usually at `~/.cache/huggingface`.

---

In [54]:
encoded_text = bpe_tokenizer(text)
print(encoded_text)

tokens_text = bpe_tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens_text)

{'input_ids': [30642, 2890, 2420, 318, 257, 4755, 4876, 286, 399, 19930, 5145], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['Token', 'izing', 'Ġtext', 'Ġis', 'Ġa', 'Ġcore', 'Ġtask', 'Ġof', 'ĠN', 'LP', 'Ġ!']


Just as with character tokenization, we can see that the words have been mapped to unique integers in the `input_ids` field. 

Spaces are converted in a special character (the Ġ) in the tokenizer prior to BPE splitting mostly to avoid digesting spaces since the standard BPE algorithm uses spaces in its process (this can seem a bit hacky but was in the original GPT2 tokenizer implementation by OpenAI).

In [55]:
print(bpe_tokenizer.convert_tokens_to_string(tokens_text))

Tokenizing text is a core task of NLP !


The `AutoTokenizer` class also has several attributes that provide information about the tokenizer. For example, we can inspect the vocabulary size and the maximum context size:

In [56]:
# Vocabulary size
print(bpe_tokenizer.vocab_size)

# Context size
print(bpe_tokenizer.model_max_length)

50257
1024


### 1.3.2 - WordPiece tokenizer

WordPiece is the tokenization algorithm Google developed to pretrain BERT. It has since been reused in quite a few Transformer models based on BERT, such as DistilBERT, MobileBERT, Funnel Transformers, and MPNET. [🤗 NLP course provides a great article explaining it](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt).

Main aspects of WordPiece:

* **Vocabulary building process**. WordPiece creates a vocabulary by splitting words into subword units. It starts with an initial vocabulary and counts word frequencies in the training corpus. The most frequent pairs are merged, and the vocabulary is updated. This process is repeated until a desired vocabulary size is reached or convergence criteria are met. Out-of-vocabulary words are handled separately.


* **Tokenization algorithm**. Tokenization differs in WordPiece and BPE in that WordPiece only saves the final vocabulary, not the merge rules learned. The vocabulary is used to split words into subword units. It starts by finding the longest subword in the vocabulary at the beginning of a word, then splits the word at that point. The process continues recursively with the remaining subword until no subword is found in the vocabulary. If a word cannot be tokenized further, it is labeled as unknown (`[UNK]`). Unlike BPE, WordPiece tokenization treats the entire word as unknown if any part of it is not in the vocabulary.

To better understand WordPiece, we are going to see it in action with DistilBERT:

In [57]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased" # it will lowercase tokenized words
wp_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

encoded_text = wp_tokenizer(text)
print(encoded_text)

tokens_text = wp_tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens_text)

{'input_ids': [101, 19204, 6026, 3793, 2003, 1037, 4563, 4708, 1997, 17953, 2361, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'token', '##izing', 'text', 'is', 'a', 'core', 'task', 'of', 'nl', '##p', '!', '[SEP]']


We can observe three things here:

* There are some special `[CLS]` and `[SEP]` tokens have been added to the start and end of the sequence. These tokens differ from model to model, but their main role is to indicate the start and end of the sequence.
* The tokens have been lowercase, which is a feature of this particular pretrained model.
* The `##` prefix in `##izing` and `##p` means that the preceding string is not whitespace. Any token with this prefix should be merged with the previous token when you convert the tokens back to a string.

In [58]:
print(wp_tokenizer.convert_tokens_to_string(tokens_text))

[CLS] tokenizing text is a core task of nlp! [SEP]


### 1.3.3 - Unigram tokenizer

The Unigram algorithm is often used in SentencePiece, which is the tokenization algorithm used by models like AlBERT, T5, mBART, Big Bird, and XLNet. [🤗 NLP course provides a great article explaining it](https://huggingface.co/learn/nlp-course/chapter6/7?fw=pt).

In contrast to BPE or WordPiece, Unigram initializes its base vocabulary with a large number of symbols, including pre-tokenized words and common substrings. The algorithm gradually reduces the vocabulary size by iteratively removing symbols based on their impact on the overall loss of a language model trained on the training data.

During training, Unigram defines a loss function, often the log-likelihood, using the current vocabulary and a unigram language model. It then evaluates the increase in loss if each symbol is removed from the vocabulary and selects a percentage (usually 10% or 20%) of symbols with the lowest increase in loss. This process continues until the desired vocabulary size is reached. The base characters are always retained to ensure any word can be tokenized.

Unlike BPE and WordPiece, Unigram does not rely on merge rules, which result in a single segmentation. Instead, multiple segmentations are sampled for each input sentence. This is achieved by selecting different sets of unigram units from the vocabulary, generating diverse subword representations for each sentence. As an example, if a trained Unigram tokenizer exhibits the vocabulary:

`["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]`

`"hugs"` could be tokenized both as `["hug", "s"]`, `["h", "ug", "s"]` or `["h", "u", "g", "s"]`. The probability of each possibility is defined by the loss the tokenizer is trained on.

Finally, there are several ways to prepare the seed vocabulary. The natural choice is to use the union of all characters and the most frequent substrings in the corpus. Frequent substrings can be enumerated in $O(T)$ time and $O(20T)$ space with the Enhanced Suffix Array algorithm ([Nong et al., 2009](https://ieeexplore.ieee.org/document/4976463)), where $T$ is the size of the corpus.

### 1.3.4 - SentencePiece tokenizer

The SentencePiece tokenizer is based on Unigram tokenization and encodes each input text as a sequence of Unicode characters. This last feature is especially useful for multilingual corpora since it allows SentencePiece to be agnostic about accents, punctuation, and the fact that many languages, like Japanese, do not have whitespace characters. Another special feature of SentencePiece is that whitespace is assigned the Unicode symbol `U+2581`, or the `▁` character, also called the lower one quarter block character. This enables SentencePiece to detokenize a sequence without ambiguities and without relying on language-specific pretokenizers. 

In our example from the previous section, for instance, we can see that WordPiece has lost the information that there is a whitespace between `"NLP"` and `"!"`. By contrast, <span style="color:blue">both BPE and SentencePiece preserve the whitespace in the tokenized text so we can convert back to the raw text without ambiguity</span>:

* SentencePience: `[..., '▁N', 'LP', '▁!']`
* BPE: `[..., 'ĠN', 'LP', 'Ġ!']`
* WordPiece: `[..., 'nl', '##p', '!']`

In [59]:
from transformers import AutoTokenizer

model_ckpt = "xlm-roberta-base" # it will lowercase tokenized words
wp_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

encoded_text = wp_tokenizer(text)
print(encoded_text)

tokens_text = wp_tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens_text)

{'input_ids': [0, 717, 1098, 84382, 7986, 83, 10, 56458, 66211, 111, 541, 37352, 711, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['<s>', '▁To', 'ken', 'izing', '▁text', '▁is', '▁a', '▁core', '▁task', '▁of', '▁N', 'LP', '▁!', '</s>']


# 2 - Tokenizer pipeline

So far we have treated tokenization as a single operation that transforms strings to integers we can pass through the model. This is not entirely accurate, and if we take a closer look we can see that it is actually a full processing pipeline that usually consists of four steps:

<table>
    <tr>
        <td><img src="images_ch2/tokenizer_pipeline.png" title="" alt="" width="500" data-align="center"></td>
    </tr>
</table>

Let's take a look at each step of the pipeline:

**Normalization**

<span style="color:blue">This step corresponds to the set of operations you apply to a raw string to make it more "cleaner"</span>. Common operations include stripping whitespace and removing accented characters. Unicode normalization is another common normalization operation applied by many tokenizers to deal with the fact that there often exist various ways to write the same character. This can make two versions of the “same” string (i.e., with the same sequence of abstract characters) appear different; Unicode normalization schemes like NFC, NFD, NFKC, and NFKD replace the various ways to write the same character with standard forms. Another example of normalization is lowercasing. If the model is expected to only accept and use lowercase characters, this technique can be used to reduce the size of the vocabulary it requires. After normalization, our example string would look like: `jack sparrow loves new york!`

**Pretokenization**

This step splits a text into smaller objects that give an upper bound to what your tokens will be at the end of training. A good way to think of this is that the pretokenizer will split your text into "words" and your final tokens will be parts of those words. For the languages that allow this (English, German, and many Indo-European languages), strings can typically be split into words on whitespace and punctuation. For example, this step might transform our `['Tokenizing', 'text', 'is', 'a', 'core', 'task', 'of', 'NLP.']`. These words are  then simpler to split into subwords with Byte-Pair Encoding (BPE) or Unigram algorithms in the next step of the pipeline. <span style="color:blue">However, splitting into "words" is not always a trivial and deterministic operation, or even an operation that makes sense. For instance, in languages like Chinese, Japanese, or Korean, grouping symbols in semantic units like Indo-European words can be a nondeterministic operation with several equally valid groups. <b>In this case, it might be best to not pretokenize the text and instead use a language-specific library for pretokenization</b></span>.

**Tokenizer model**
    
Once the input texts are normalized and pretokenized, the tokenizer applies a subword splitting model on the words. <span style="color:blue">This is the part of the pipeline that needs to be trained on your corpus (or that has been trained if you are using a pretrained tokenizer). <b>The role of the tokenizer is to split the words into subwords to reduce the size of the vocabulary and try to reduce the number of out-of-vocabulary tokens</b></span>. Several subword tokenization algorithms exist, including BPE, Unigram, and WordPiece. For instance, our running example might look like `['token', 'izing', 'text', 'is', 'a', 'core', 'task', 'of', 'nl', 'p', '.']` after the tokenizer model is applied. 

Note that at this point we no longer have a list of strings but a list of integers (input IDs); to keep the example illustrative, we’ve kept the words but dropped the quotes to indicate the transformation.

**Post-processing**
    
<span style="color:blue">This is the last step of the tokenization pipeline, in which some additional transformations can be applied on the list of tokens. For instance, by adding special tokens at the beginning or end of the input sequence of token indices</span>. For example, a BERT-style tokenizer would add classifications and separator tokens: `['[CLS]', 'token', '##izing', 'text', 'is', 'a', 'core', 'task', 'of', 'nl', '##p', '.', '[SEP]']`. This sequence (recall that this will be a sequence of integers, not the tokens you see here) can then be fed to the model.